# Advent of Code


In [1]:
import numpy as np
import re
import pandas as pd
import itertools
from tqdm import tqdm
import sys
import matplotlib.pyplot as plt
%matplotlib inline

### Day 1: Not Quite Lisp

In [ ]:
with open('./data/input1.txt', 'r') as f:
    data = f.next()
ups_and_downs = np.array([1 if x=='(' else -1 for x in data])
print ups_and_downs.sum()
print np.where(ups_and_downs.cumsum()==-1)[0][0] + 1

### Day 2: I Was Told There Would Be No Math

In [ ]:
with open('./data/input2.txt', 'r') as f:
    dims = np.array([map(str.strip, spec.split('x')) for spec in f], dtype=int)
l, w, h = dims.T
side_areas = np.array([l*w, w*h, h*l])
surf_areas = 2 * side_areas.sum(0)
print (surf_areas + side_areas.min(0)).sum()
print sum(((2 * sum(sorted(spec)[:2])) + spec.prod() for spec in dims))

### Day 3: Perfectly Spherical Houses in a Vacuum

In [ ]:
with open('./data/input3.txt', 'r') as f:
    data = f.next()
translation_map = {'^':(0,1), '>':(1,0), 'v':(0,-1), '<':(-1,0)}
# don't forget to include the starting location (0,0) twice, once for Santa and once for Robo-Santa!
translations = np.array([(0,0), (0,0)] + [translation_map[d] for d in data])
santa, robosanta = translations[::2], translations[1::2]
def unique_locations(translations):
    path = translations.cumsum(0)
    return set(map(tuple, path))

print len(unique_locations(translations))
print len(set.union(unique_locations(santa), unique_locations(robosanta)))

### Day 4: The Ideal Stocking Stuffer

In [ ]:
import hashlib as hl

def find_hash(key, min_zeros):
    for i in itertools.count():
        h = hl.md5(key + str(i))
        if h.hexdigest()[:min_zeros] == '0'*min_zeros:
            break
    return i

print find_hash('ckczppom', 5)
print find_hash('ckczppom', 6)

### Day 5: Doesn't He Have Intern-Elves For This?

In [ ]:
with open('./data/input5.txt', 'r') as f:
    strings = [s.strip() for s in f]

vowels_cond = np.array(map(lambda x: [c in 'aeiou' for c in x], strings)).sum(1) > 2
repeat_cond = np.array([len(re.findall('(\\w)\\1', x)) for x in strings]) > 0
forbid_cond = np.array(map(lambda x: [s in x for s in ('ab', 'cd', 'pq', 'xy')], strings)).sum(1) == 0
repeat_cond2 = np.array([len(re.findall('(\\w{2}).*\\1', x)) for x in strings]) > 0
repeat_cond3 = np.array([len(re.findall('(\\w).\\1', x)) for x in strings]) > 0

print (forbid_cond & repeat_cond & vowels_cond).sum()
print (repeat_cond2 & repeat_cond3).sum()

### Day 6: Probably a Fire Hazard

In [ ]:
lights = np.zeros((1000, 1000), dtype=bool)
display = np.zeros((1000, 1000), dtype=int)
with open('./data/input6.txt', 'r') as f:
    coords = np.array([re.findall('(.*) (\d+),(\d+).*?(\d+),(\d+)', r)[0] for r in f])

commands, (x1, y1, x2, y2) = coords.T[0], coords.T[1:].astype(int)
for i, c in enumerate(commands):
    if c == 'turn on':
        lights[x1[i]:x2[i]+1, y1[i]:y2[i]+1] = True
        display[x1[i]:x2[i]+1, y1[i]:y2[i]+1] += 1
    elif c == 'turn off':
        lights[x1[i]:x2[i]+1, y1[i]:y2[i]+1] = False
        display[x1[i]:x2[i]+1, y1[i]:y2[i]+1] -= 1
    else:
        lights[x1[i]:x2[i]+1, y1[i]:y2[i]+1] = ~lights[x1[i]:x2[i]+1, y1[i]:y2[i]+1]
        display[x1[i]:x2[i]+1, y1[i]:y2[i]+1] += 2
    display = np.maximum(0, display)

print lights.sum()
print np.maximum(0, display).sum()

### Day 7: Some Assembly Required

In [ ]:
d = {'AND':'&', 'OR':'|', 'RSHIFT':'>>', 'LSHIFT':'<<', 'NOT':'~'}
bitwise = re.compile(r'(' + '|'.join(d.keys()) + r')')

with open('./data/input7.txt', 'r') as f:
    data = f.read().replace('if', 'iff').replace('as', 'ass').replace('in', 'inn').replace('is', 'iss').split('\n')[:-1]

data = map(lambda x: bitwise.sub(lambda y: d[y.group()], x), data)  # replace by bit-wise operators
data = map(lambda x: re.findall('(.+) -> (.+)', x)[0], data)
circuit = {x[1]: x[1] + ' = ' + x[0] for x in data} # 

def run_circuit(circuit, out):
    '''Not efficient: Runs in O(N**2)'''
    for _ in range(len(circuit)):
        for wire in circuit:
            try: exec(circuit[wire])
            except: pass
    return eval(out)

a = run_circuit(circuit, 'a')
circuit['b'] = 'b = %d' % a  # override b signal
print a
print run_circuit(circuit, 'a')  # run updated circuit

###  Day 8: Matchsticks

In [ ]:
with open('./data/input8.txt', 'r') as f:
    data = f.read().split('\n')[:-1]

escapechars = re.compile('|'.join([r'\\', "'", '"']))

df = pd.DataFrame({'stringcode': data, 'string': map(eval, data),
                   'stringcodecode': map(lambda x: "'"+escapechars.sub(lambda y:'\\'+y.group(),x)+"'", data)
                  })
print (df.stringcode.map(len) - df.string.map(len)).sum()
print (df.stringcodecode.map(len) - df.stringcode.map(len)).sum()
df.head()

### Day 9: All in a Single Night

In [ ]:
with open('./data/input9.txt', 'r') as f:
    data = map(lambda x: x.split(' ')[::2], f.read().split('\n')[:-1])

df = pd.DataFrame(data, columns=['A', 'B', 'distance']).pivot('A', 'B', 'distance')

def dist(A, B):
    try: return int(df[A][B])
    except: return int(df[B][A])

min_tour_len = np.Inf
max_tour_len = 0
locations = df.columns | df.index
for tour in itertools.permutations(locations, len(locations)):
    tour_len = sum([dist(tour[j], tour[j+1]) for j in xrange(len(tour)-1)])
    min_tour_len = min(tour_len, min_tour_len)
    max_tour_len = max(tour_len, max_tour_len)

print min_tour_len
print max_tour_len

### Day 10: Elves Look, Elves Say

In [ ]:
def look_and_say(N):
    new_N = ''
    last_char = N[0]
    count = 0
    for c in N:
        if c == last_char:
            count += 1
        else:
            new_N += str(count) + last_char
            last_char = c
            count = 1
    new_N += str(count) + last_char
    return new_N

N = '1113122113'
lengths = []
for _ in xrange(50):
    N = look_and_say(N)
    lengths.append(len(N))
print lengths[39]
print lengths[-1]

### Day 11: Corporate Policy

In [ ]:
def next_pwd(pwd):
    return next_pwd(pwd[:-1]) + 'a' if pwd[-1] == 'z' else pwd[:-1] + unichr(ord(pwd[-1]) + 1)

def check_for_staight(pwd):
    for i, c in enumerate(pwd[:-2]):
        if unichr(ord(c)+1) == pwd[i+1] and unichr(ord(c)+2) == pwd[i+2]:
            return True
    return False

def check_for_chars(pwd):
    return ~np.array([c in pwd for c in 'iol']).any()

def check_for_pairs(pwd):
    return len(re.findall('(\w)\\1', pwd)) > 1

def next_valid_pwd(pwd):
    for _ in itertools.count():
        if check_for_staight(pwd) and check_for_chars(pwd) and check_for_pairs(pwd):
            return pwd
        else:
            pwd = next_pwd(pwd)

pwd2 = next_valid_pwd('cqjxjnds')
print pwd2
print next_valid_pwd(next_pwd(pwd2))

### Day 12: JSAbacusFramework.io

In [ ]:
with open('./data/input12.txt', 'r') as f:
    data = eval(f.read())

def sum_nested_numerals(obj, exclude_color=None):
    s = 0
    if type(obj) is dict and exclude_color in obj.itervalues():
        return s
    elif type(obj) is dict:
        for key in obj:
            s += sum_nested_numerals(obj[key], exclude_color)
    elif type(obj) is list:
        for item in obj:
            s += sum_nested_numerals(item, exclude_color)
    elif type(obj) is int:
        s += obj
    return s

print sum_nested_numerals(data)
print sum_nested_numerals(data, exclude_color='red')

### Day 13: Knights of the Dinner Table

In [ ]:
with open('./data/input13.txt', 'r') as f:
    data = map(lambda x: np.array(x.replace('gain ', '+').replace('lose ', '-').split(' '))[[0,2,-1]], f.read().split('.\n')[:-1])

w = pd.DataFrame(np.array(zip(*data)).T, columns=['A','weight','B']).pivot('A','B','weight').fillna(0).astype(int)
w['You'] = 0
w.loc['You'] = 0

def max_seating_score(w):
    max_score = 0
    for seats in itertools.permutations(w.index, w.shape[0]):
        score = sum([w[seats[j]][seats[j+1]] for j in xrange(-1, len(seats)-1)])
        max_score = max(score, max_score)
    return max_score

print max_seating_score((w + w.T).iloc[:-1, :-1])
print max_seating_score(w + w.T)

### Day 14: Reindeer Olympics

In [ ]:
with open('./data/input14.txt', 'r') as f:
    data = np.array(map(lambda x: np.array(x.split(' '))[[3,6,-2]], f.read().split('\n')[:-1]), dtype=int)

def state_in_time(speed, run_duration, rest_duration):
    cycles = int(np.ceil(float(time) / (run_duration + rest_duration)))
    return (([1]*run_duration + [0]*rest_duration) * cycles)

time = 2503
cum_distances = np.array(map(lambda x: np.cumsum(state_in_time(*x)[:time]) * x[0], data))

print max(cum_distances[:, -1])
print pd.Series(cum_distances.argmax(0)).value_counts().max()

### Day 15: Science for Hungry People

In [ ]:
with open('./data/input15.txt', 'r') as f:
    # properties x ingredients
    properties = np.reshape(re.findall('-?\d+', f.read()), (4, 5)).astype(int).T

max_score = 0
max_score_lite = 0
for c in itertools.product(*[xrange(101)]*3):
    c = c[0], c[1], c[2], 100 - sum(c)
    if c[-1] >= 0:  # do the teaspoons add to 100?
        mix = np.maximum(0, np.dot(properties, c))
        max_score = max(max_score, mix[:-1].prod())
        if mix[-1] == 500:  # does this recipe yield 500 calories?
            max_score_lite = max(max_score_lite, mix[:-1].prod())

print max_score
print max_score_lite

### Day 16: Aunt Sue

In [ ]:
df = pd.DataFrame(columns=['children', 'cats', 'samoyeds', 'pomeranians', 'akitas', 'vizslas',
                           'goldfish', 'trees', 'cars', 'perfumes'])
with open('./data/input16.txt', 'r') as f:
    data = f.read().split('\n')[:-1]

aunt_signature = np.array([3, 7, 2, 3, 0, 0, 5, 3, 2, 1])
for r in tqdm(data):
    exec('df.loc[%s,:] = {\'%s\':%s, \'%s\':%s, \'%s\':%s}' % re.findall('(\d+): (.+): (\d+), (.+): (\d+), (.+): (\d+)', r)[0])
    
print df[df.apply(lambda x: sum(x == aunt_signature) == 3, axis=1)]
print df[df.apply(lambda x:
         sum(list(x[[0,2,4,5,8,9]]==aunt_signature[[0,2,4,5,8,9]]) +
         list(x[[1,7]]>aunt_signature[[1,7]])+
         list(x[[3,6]]<aunt_signature[[3,6]])) == 3,
         axis=1)]

### Day 17: No Such Thing as Too Much

In [ ]:
with open('./data/input17.txt', 'r') as f:
    containers = np.array(f.read().split('\n')[:-1], dtype=int)

count = 0
combo_150 = []
for n in tqdm(xrange(1, len(containers) + 1)):
    for combo in itertools.combinations(containers, n):
        if sum(combo) == 150:
            count += 1
            combo_150.append(combo)

print count
print pd.Series(map(len, combo_150)).value_counts().iloc[-1]

### Day 18: Like a GIF For Your Yard

In [ ]:
with open('./data/input18.txt', 'r') as f:
    lights = np.array(map(list, f.read().replace('#', '1').replace('.', '0').split('\n')[:-1]), dtype=int)
    lights2 = lights.copy()
    lights2[[0, 0, 99, 99], [0, 99, 0, 99]] = 1 
    
def next_state(lights):
    n = lights.shape[0]
    appended_lights = np.zeros((n + 2, n + 2))
    appended_lights[1:-1, 1:-1] = lights
    new_state = np.zeros_like(appended_lights)
    for i, j in itertools.product(xrange(1, n + 1), xrange(1, n + 1)):
        s = appended_lights[i - 1:i + 2, j - 1:j + 2].sum() - appended_lights[i, j]
        if appended_lights[i, j] == 1:
            new_state[i, j] = 0 if s not in [2, 3] else 1
        else:
            new_state[i, j] = 1 if s == 3 else 0
    return new_state[1:-1, 1:-1]

# plt.figure(figsize=(7,7))
for i in tqdm(range(100)):
    lights = next_state(lights)
    lights2 = next_state(lights2)
    lights2[[0, 0, 99, 99], [0, 99, 0, 99]] = 1
    
#     plt.imshow(lights, interpolation='nearest', cmap=plt.cm.Blues)
#     plt.axis('off')
#     plt.savefig('frame%03d.png' % i)
#     plt.cla()
# !convert -delay 5 *png animated.gif  # requires Imagemagick

print lights.sum()
print lights2.sum()

### Day 19: Medicine for Rudolph

In [171]:
with open('./data/input19.txt') as f:
    data = f.read().split('\n')[:-1]
repl = map(lambda x: re.findall('(.+) => (.*)', x)[0], data[:-2])
repl_r = dict(zip(*zip(*repl)[::-1]))
molecule = data[-1]
repl = {key:[v for k, v in repl if k == key] for key in zip(*repl)[0]}

def derivative_molecules(molecule, repl):
    new_molecules = []
    for k, V in repl.iteritems():
        n_matches = len(re.findall(k, molecule))
        for v in V:
            new_molecules.extend([re.sub('^((.*?%s){%d}.*?)%s' % (k, i, k), '\\1'+v, molecule) for i in xrange(n_matches)])
    return new_molecules

def reverse_engineer(molecule):
    molecules = [molecule]
    while molecules[-1] != 'e':
        molecules.append(re.sub('^(.*)(' + '|'.join(repl_r.keys()) + ')(.*?)$',
                                lambda x: x.group(1) + repl_r[x.group(2)] + x.group(3),
                                molecules[-1]))
    return molecules

print len(set(derivative_molecules(molecule, repl)))
print len(reverse_engineer(molecule)) - 1

535
212


### Day 20: Infinite Elves and Infinite Houses

In [506]:
def prime_sieve(limit):
    """source: http://stackoverflow.com/a/3941967/5350602"""
    a = np.array([True] * limit)  # Initialize the primality list
    a[0] = a[1] = False

    for (i, isprime) in enumerate(a):
        if isprime:
            yield i
            a[i**2:limit:i] = False  # Mark factors non-prime

def prime_factorization(n, prime_sieve):
    """source: https://en.wikipedia.org/wiki/Trial_division"""
    if n < 2:
        return []
    prime_factors = []
    for p in prime_sieve(int(n**0.5) + 1):
        if p*p > n: break
        while n % p == 0:
            prime_factors.append(p)
            n /= p
    if n > 1:
        prime_factors.append(n)
    return prime_factors

def factors(n, limit=1):
    fa = []
    prime_factors = prime_factorization(n, prime_sieve)
    for l in xrange(1, len(prime_factors) + 1):
        fa.extend([np.array(c).prod() for c in itertools.combinations(prime_factors, l)])
    fa = np.unique(fa + [1])
    if limit > 1:
        fa = fa[fa >= (n / limit)]
    return fa


data = 34000000
for n in itertools.count(785000):
    if factors(n).sum() * 10 >= data:
        print n
        break
for n in itertools.count(830000):
    if factors(n, limit=50.0).sum() * 11 >= data:
        print n
        break

786240
831600


### Day 21: RPG Simulator 20XX

In [837]:
weapons = np.array(re.findall('\d+',
            """Weapons:    Cost  Damage  Armor
               Dagger        8     4       0
               Shortsword   10     5       0
               Warhammer    25     6       0
               Longsword    40     7       0
               Greataxe     74     8       0"""), dtype=int).reshape(5, 3)

armors = np.array(re.findall('\d+',
            """Armor:      Cost  Damage  Armor
               noarmor       0     0       0
               Leather      13     0       1
               Chainmail    31     0       2
               Splintmail   53     0       3
               Bandedmail   75     0       4
               Platemail   102     0       5"""), dtype=int).reshape(6, 3)

rings = np.array(re.findall(' \d+',
            """Rings:      Cost  Damage  Armor
               noring        0     0       0
               noring        0     0       0
               Damage +1    25     1       0
               Damage +2    50     2       0
               Damage +3   100     3       0
               Defense +1   20     0       1
               Defense +2   40     0       2
               Defense +3   80     0       3"""), dtype=int).reshape(8, 3)

ring_combo = itertools.combinations(rings, 2)
combo_stats = pd.DataFrame([w + a + sum(r) for (w, a, r) in itertools.product(weapons, armors, ring_combo)],
                           columns=['cost','damage', 'armor'])
    
def duel(player_stats, boss_stats, player_turn=True, verbose=False):
    boss_hp = boss_stats[0]
    player_hp = player_stats[0]
    while player_hp > 0 and boss_hp > 0:
        if player_turn:
            boss_hp -= max(1, player_stats[1] - boss_stats[2])
            if verbose:
                print 'The player deals %d-%d = %d damage; the boss goes down to %d hit points.' % \
                (player_stats[1], boss_stats[2], player_stats[1] - boss_stats[2], boss_hp)
        else:
            player_hp -= max(1, boss_stats[1] - player_stats[2])
            if verbose:
                print 'The boss deals %d-%d = %d damage; the player goes down to %d hit points.' % \
                (boss_stats[1], player_stats[2], boss_stats[1] - player_stats[2], player_hp)
        player_turn = not player_turn
    return (not player_turn)

boss_stats = [103, 9, 2]  # hit_points, damage, armor
combo_stats['player_wins'] = combo_stats.apply(lambda x: duel([100, x[1], x[2]], boss_stats, player_turn=True), axis=1)

print combo_stats[combo_stats.player_wins].sort('cost').iloc[0]  # part 1
print combo_stats[~combo_stats.player_wins].sort('cost').iloc[-1]  # part 2

cost            121
damage            9
armor             2
player_wins    True
Name: 568, dtype: object
cost             201
damage             7
armor              4
player_wins    False
Name: 52, dtype: object


### Day 22: Wizard Simulator 20XX

In [108]:
class character(object):
    
    def __init__(self, mana=0, hp=0, damage=0, armor=0):
        self.mana, self.hp, self.damage, self.armor = mana, hp, damage, armor
        self._effects = {}

    def attack(self, target, verbose):  # Physical attack
        if verbose: print 'Boss attacks for %d - %d = %d damage!' % (self.damage, target.armor, self.damage - target.armor)
        target.hp -= max(1, self.damage - target.armor)  # at least 1 dmg must be dealt
        
    def apply_effects(self, verbose):
        for e in self._effects.keys():
            s = eval('self._' + e + '()')
            self._effects[e] -= 1
            if verbose: print s, 'its timer is now %d.' % self._effects[e]
            if self._effects[e] == 0:  # remove effect when expired
                self._effects.pop(e)
                if verbose: print '%s wears off.' % e[:-7].title()

    def _shield_effect(self):
        if self._effects['shield_effect'] == 1: self.armor -= 7
        return 'Shield is active;'
    def _poison_effect(self):
        self.hp -= 3
        return 'Poisson deals 3 damage;'
    def _recharge_effect(self):
        self.mana += 101
        return 'Recharge provides 101 mana;'


class wizard(character):
    
    def __init__(self, *args, **kwargs):
        character.__init__(self, *args, **kwargs)
        self.mana_spend = 0
        self.spells = {'magic_missile': (self.magic_missile, 53), 'drain': (self.drain, 73),
                       'shield': (self.shield, 113), 'poison': (self.poison, 173), 'recharge': (self.recharge, 229)}
            
    def can_cast(self, spell, target):
        return (self.mana >= self.spells[spell][1]) and (spell + '_effect' not in target._effects.keys() + self._effects.keys())
    
    def cast(self, spell, target, verbose):  # Magical attack
        if self.can_cast(spell, target):
            if verbose: print 'Player casts %s.' % spell.title().replace('_', ' ')
            self.mana -= self.spells[spell][1]
            self.mana_spend += self.spells[spell][1]
            self.spells[spell][0](target)
            return True
        
    def magic_missile(self, target):  # Magic Missile (53) instantly does 4 damage.
        target.hp -= 4
    def drain(self, target):  # Drain (73) instantly does 2 damage and heals you for 2 hit points.
        target.hp -= 2
        self.hp += 2
    def shield(self, target):  # Shield (113, 6 turns) while active, increases your armor by 7.
        self.armor += 7
        self._effects['shield_effect'] = 6
    def poison(self, target):  # Poison (173, 6 turns) while active, at the start of each turn deals 3 damage.
        target._effects['poison_effect'] = 6
    def recharge(self, target):  # Recharge (229, 5 turns) while active, at the start of each turn gives you 101 mana.
        self._effects['recharge_effect']  = 5


def duel(player, boss, spell_seq, player_turn=True, verbose=True, hardmode=False, mana_thres=1e6):

    spell_iter = itertools.cycle(spell_seq)
    while True:
        
        if verbose:
            print '\n-- %s turn --' % ['Boss', 'Player'][player_turn]
            print '- Player has %d hit points, %d armor, %d mana' % (player.hp, player.armor, player.mana)
            print '- Boss has %d hit points' % boss.hp
        
        if hardmode and player_turn:
            player.hp -= 1
            if verbose: print 'Hard mode deals 1 damage.'
        
        player.apply_effects(verbose)  # Check for effects.
        if player.hp <= 0:
            if verbose: print 'Boss wins.'
            return False

        boss.apply_effects(verbose)
        if boss.hp <= 0:
            if verbose: print 'Player wins.'
            return True
        
        if player_turn:
            if player.mana < 53:
                if verbose: print 'Player mana depleted. Boss wins.'
                return False            
            cast_fail_count = 0
            while not player.cast(spell_iter.next(), boss, verbose):
                cast_fail_count += 1
                if cast_fail_count == len(spell_seq): return False
        else: boss.attack(player, verbose)
            
        if player.mana_spend > mana_thres: return False  # for pruning
        
        if boss.hp <= 0:
            if verbose: print 'Player wins.'
            return True
        elif player.hp <= 0:
            if verbose: print 'Boss wins.'
            return False
    
        player_turn = not player_turn
        
        
actions = ['shield', 'magic_missile', 'recharge', 'drain', 'poison']
min_mana = 9999
for l in range(1, 6):
    for combo in itertools.product(*[actions]*l):
        player = wizard(mana=500, hp=50)
        boss = character(hp=55, damage=8, armor=0)
        player_won = duel(player, boss, combo, verbose=False, mana_thres=min_mana) 
        if player_won and player.mana_spend < min_mana:
            min_mana = player.mana_spend
            print min_mana, combo
            sys.stdout.flush()

print
min_mana = 9999
for l in range(1, 30):
    for combo in np.random.choice(actions, size=(1000, l)):  # itertools.product(*[actions]*l):
        player = wizard(mana=500, hp=50);
        boss = character(hp=55, damage=8, armor=0)
        player_won = duel(player, boss, combo, verbose=False, hardmode=True, mana_thres=min_mana)
        if player_won and player.mana_spend < min_mana:
            min_mana = player.mana_spend
            print min_mana, combo
            sys.stdout.flush()

5301 ('shield', 'magic_missile', 'recharge')
2060 ('shield', 'recharge', 'poison')
1947 ('recharge', 'poison', 'shield')
1718 ('poison', 'recharge', 'shield')
1704 ('shield', 'recharge', 'poison', 'magic_missile')
1651 ('recharge', 'poison', 'shield', 'magic_missile')
1591 ('recharge', 'poison', 'magic_missile', 'shield')
1422 ('poison', 'recharge', 'shield', 'magic_missile')
1362 ('poison', 'recharge', 'magic_missile', 'shield')
1348 ('magic_missile', 'magic_missile', 'recharge', 'shield', 'poison')
1295 ('magic_missile', 'recharge', 'shield', 'poison', 'magic_missile')
953 ('poison', 'magic_missile', 'recharge', 'magic_missile', 'shield')

1947 ['recharge' 'poison' 'shield']
1718 ['poison' 'recharge' 'shield']
1591 ['recharge' 'poison' 'magic_missile' 'shield']
1495 ['poison' 'recharge' 'drain' 'shield']
1422 ['poison' 'recharge' 'shield' 'magic_missile']
1362 ['poison' 'recharge' 'poison' 'magic_missile' 'shield']
1309 ['poison' 'recharge' 'shield' 'poison' 'magic_missile']
1289 ['p

### Day 23: Opening the Turing Lock

In [ ]:
# from PIL import Image
# im = Image.open("./../../../Desktop/Untitled.png")
# lights = ~np.array(im).astype(bool)

# def next_state(lights):
#     n = lights.shape[0]
#     appended_lights = np.zeros((n + 2, n + 2))
#     appended_lights[1:-1, 1:-1] = lights
#     new_state = np.zeros_like(appended_lights)
#     for i, j in itertools.product(xrange(1, n + 1), xrange(1, n + 1)):
#         s = appended_lights[i - 1:i + 2, j - 1:j + 2].sum() - appended_lights[i, j]
#         if appended_lights[i, j] == 1:
#             new_state[i, j] = 0 if s not in [2, 3] else 1
#         else:
#             new_state[i, j] = 1 if s == 3 else 0
#     return new_state[1:-1, 1:-1]

# plt.figure(figsize=(10, 10))
# plt.imshow(lights, interpolation='nearest', cmap=plt.cm.Blues)
# plt.axis('off')
# for i in tqdm(range(150, 100, -1)):
#     plt.savefig('frame%03d.png' % i, dpi=100)

# for i in tqdm(range(100, 0, -1)):
#     lights = next_state(lights)
#     plt.imshow(lights, interpolation='nearest', cmap=plt.cm.Blues)
#     plt.axis('off')
#     plt.savefig('frame%03d.png' % i, dpi=100)
#     plt.cla()

# # !convert -delay 5 *png animated.gif  # requires Imagemagick